<a href="https://colab.research.google.com/github/HiranmaiKaredla/PatriotHacks/blob/main/Text_to_CSV_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = ""

In [2]:
#!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv

In [3]:
import pandas as pd

df = pd.read_csv("/content/apartments_for_rent_classified_10K.csv", encoding="latin-1", sep=";")
print(df.shape)
print(df.columns.tolist())

(10000, 22)
['id', 'category', 'title', 'body', 'amenities', 'bathrooms', 'bedrooms', 'currency', 'fee', 'has_photo', 'pets_allowed', 'price', 'price_display', 'price_type', 'square_feet', 'address', 'cityname', 'state', 'latitude', 'longitude', 'source', 'time']


In [4]:
df.head(5)

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668626895,housing/rent/apartment,"Studio apartment 2nd St NE, Uhland Terrace NE,...","This unit is located at second St NE, Uhland T...",NaN,NaN,0.0,USD,No,Thumbnail,...,$790,Monthly,101,NaN,Washington,DC,38.9057,-76.9861,RentLingo,1577359415
1,5664597177,housing/rent/apartment,Studio apartment 814 Schutte Road,"This unit is located at 814 Schutte Road, Evan...",NaN,NaN,1.0,USD,No,Thumbnail,...,$425,Monthly,106,814 Schutte Rd,Evansville,IN,37.9680,-87.6621,RentLingo,1577017063
2,5668626833,housing/rent/apartment,"Studio apartment N Scott St, 14th St N, Arling...","This unit is located at N Scott St, 14th St N,...",NaN,1.0,0.0,USD,No,Thumbnail,...,"$1,390",Monthly,107,NaN,Arlington,VA,38.8910,-77.0816,RentLingo,1577359410
3,5659918074,housing/rent/apartment,Studio apartment 1717 12th Ave,"This unit is located at 1717 12th Ave, Seattle...",NaN,1.0,0.0,USD,No,Thumbnail,...,$925,Monthly,116,1717 12th Avenue,Seattle,WA,47.6160,-122.3275,RentLingo,1576667743
4,5668626759,housing/rent/apartment,"Studio apartment Washington Blvd, N Cleveland ...","This unit is located at Washington Blvd, N Cle...",NaN,NaN,0.0,USD,No,Thumbnail,...,$880,Monthly,125,NaN,Arlington,VA,38.8738,-77.1055,RentLingo,1577359401


In [6]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/7

In [41]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchain-openai faiss-cpu

In [7]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///properties.db")
df.to_sql("properties", engine, index=False)

10000

In [8]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM properties WHERE cityname = 'Seattle' limit 3;")

sqlite
['properties']


"[(5659918074, 'housing/rent/apartment', 'Studio apartment 1717 12th Ave', 'This unit is located at 1717 12th Ave, Seattle, 98122, WAMonthly rental rates range from $925We have studio units available for rent', None, 1.0, 0.0, 'USD', 'No', 'Thumbnail', None, 925, '$925', 'Monthly', 116, '1717 12th Avenue', 'Seattle', 'WA', 47.616, -122.3275, 'RentLingo', 1576667743), (5668634310, 'housing/rent/apartment', 'Studio apartment 6921 Roosevelt Way NE', 'This unit is located at 6921 Roosevelt Way NE, Seattle, 98115, WAMonthly rental rates range from $1195We have studio units available for rent', None, 1.0, 0.0, 'USD', 'No', 'Thumbnail', 'Cats,Dogs', 1195, '$1,195', 'Monthly', 223, '6921 Roosevelt Way NE', 'Seattle', 'WA', 47.616, -122.3275, 'RentLingo', 1577359975), (5668640879, 'housing/rent/apartment', 'Studio apartment 714 N. 95th St.', 'This unit is located at 714 N. 95th St., Seattle, 98103, WAMonthly rental rates range from $1150We have studio units available for rent', None, 1.0, 0.0, 

In [36]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True, return_intermediate_steps=True)

In [37]:
result = agent_executor.invoke({"input": "Give all the properties in the Arlington with 2bhk"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


properties
Invoking: `sql_db_schema` with `{'table_names': 'properties'}`



CREATE TABLE properties (
	id BIGINT, 
	category TEXT, 
	title TEXT, 
	body TEXT, 
	amenities TEXT, 
	bathrooms FLOAT, 
	bedrooms FLOAT, 
	currency TEXT, 
	fee TEXT, 
	has_photo TEXT, 
	pets_allowed TEXT, 
	price BIGINT, 
	price_display TEXT, 
	price_type TEXT, 
	square_feet BIGINT, 
	address TEXT, 
	cityname TEXT, 
	state TEXT, 
	latitude FLOAT, 
	longitude FLOAT, 
	source TEXT, 
	time BIGINT
)

/*
3 rows from properties table:
id	category	title	body	amenities	bathrooms	bedrooms	currency	fee	has_photo	pets_allowed	price	price_display	price_type	square_feet	address	cityname	state	latitude	longitude	source	time
5668626895	housing/rent/apartment	Studio apartment 2nd St NE, Uhland Terrace NE, Washington, DC 20002	This unit is located at second St NE, Uhland Terrace NE, Washington, DC 20002, Washington, 20002, DC	None	None	0.0

In [38]:
import json
result["output"]

'Here are some properties in Arlington with 2 bedrooms:\n\n1. Property: Studio apartment 815 18th St S\n   - Address: 815 18th St S, Arlington, VA 22202\n   - Price: $1,549 per month\n   - Amenities: On-Site Laundry, Dishwasher, Fitness facilities, Refrigerator\n\n2. Property: Studio apartment 1919 Clarendon Blvd\n   - Address: 1919 Clarendon Blvd, Arlington, VA 22201\n   - Price: $2,100 per month\n   - Amenities: Balcony, Deck, Patio, A/c, Dishwasher, Refrigerator, Fitness facilities\n\n3. Property: Studio apartment 1351 N Rolfe St\n   - Address: 1351 N Rolfe St, Arlington, VA 22209\n   - Price: $2,150 per month\n   - Amenities: Business facility, Fitness facilities, Air conditioned, Pool\n\n4. Property: Studio apartment 505 18th Street South\n   - Address: 505 18th Street South, Arlington, VA 22202\n   - Price: $1,953 per month\n   - Amenities: On Bus Line, Business facility, Fitness facilities, Sheltered parking\n\n5. Property: Two BR 308 S W St\n   - Address: 308 S W St, Arlington,

In [39]:
result


{'input': 'Give all the properties in the Arlington with 2bhk',
 'output': 'Here are some properties in Arlington with 2 bedrooms:\n\n1. Property: Studio apartment 815 18th St S\n   - Address: 815 18th St S, Arlington, VA 22202\n   - Price: $1,549 per month\n   - Amenities: On-Site Laundry, Dishwasher, Fitness facilities, Refrigerator\n\n2. Property: Studio apartment 1919 Clarendon Blvd\n   - Address: 1919 Clarendon Blvd, Arlington, VA 22201\n   - Price: $2,100 per month\n   - Amenities: Balcony, Deck, Patio, A/c, Dishwasher, Refrigerator, Fitness facilities\n\n3. Property: Studio apartment 1351 N Rolfe St\n   - Address: 1351 N Rolfe St, Arlington, VA 22209\n   - Price: $2,150 per month\n   - Amenities: Business facility, Fitness facilities, Air conditioned, Pool\n\n4. Property: Studio apartment 505 18th Street South\n   - Address: 505 18th Street South, Arlington, VA 22202\n   - Price: $1,953 per month\n   - Amenities: On Bus Line, Business facility, Fitness facilities, Sheltered park

In [42]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///properties.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM properties LIMIT 10;")

sqlite
['properties']


"[(5668626895, 'housing/rent/apartment', 'Studio apartment 2nd St NE, Uhland Terrace NE, Washington, DC 20002', 'This unit is located at second St NE, Uhland Terrace NE, Washington, DC 20002, Washington, 20002, DCMonthly rental rates range from $790 - $1090We have studio units available for rent', None, None, 0.0, 'USD', 'No', 'Thumbnail', None, 790, '$790', 'Monthly', 101, None, 'Washington', 'DC', 38.9057, -76.9861, 'RentLingo', 1577359415), (5664597177, 'housing/rent/apartment', 'Studio apartment 814 Schutte Road', 'This unit is located at 814 Schutte Road, Evansville, 47712, INMonthly rental rates range from $425 - $445We have studio - 1 beds units available for rent', None, None, 1.0, 'USD', 'No', 'Thumbnail', None, 425, '$425', 'Monthly', 106, '814 Schutte Rd', 'Evansville', 'IN', 37.968, -87.6621, 'RentLingo', 1577017063), (5668626833, 'housing/rent/apartment', 'Studio apartment N Scott St, 14th St N, Arlington, VA 22209', 'This unit is located at N Scott St, 14th St N, Arlingto

In [43]:
%pip install -qU langchain-openai

In [65]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)


'SELECT "id" \nFROM properties \nWHERE cityname = \'Arlington\' AND bedrooms = 2\nLIMIT 5;'

In [75]:
query = "Give all the properties in seatle with 2bhk which has parking"
response = chain.invoke({"question": "Get ids of {query}".format(query=query)})
response

'SELECT "id"\nFROM properties\nWHERE cityname = \'Seattle\' AND bedrooms = 2 AND amenities LIKE \'%parking%\'\nLIMIT 5;'

In [76]:
records = db.run(response)

In [77]:
records

'[(5668639228,)]'

In [72]:
import re
id_numbers = [int(num) for num in re.findall(r'\d+', records)]
print(id_numbers)

[5668624600, 5668613886, 5668613880, 5668613955, 5509079183]


In [69]:
print(json.dumps(records, indent=2, default=str))

"[(5668624600,), (5668613886,), (5668613880,), (5668613955,), (5509079183,)]"


In [74]:
filtered_df = df[df['id'].isin(id_numbers)]
filtered_df

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
587,5668624600,housing/rent/apartment,Studio apartment 815 18th St S,"This unit is located at 815 18th St S, Arlingt...","Dishwasher,Refrigerator",1.0,2.0,USD,No,Thumbnail,...,"$1,549",Monthly,431,815 18th St S,Arlington,VA,38.8738,-77.1055,RentLingo,1577359252
1244,5668613886,housing/rent/apartment,Studio apartment 1919 Clarendon Blvd,"This unit is located at 1919 Clarendon Blvd, A...","Cable or Satellite,Dishwasher,Internet Access,...",1.0,2.0,USD,No,Thumbnail,...,"$2,100",Monthly,533,1919 Clarendon Boulevard,Arlington,VA,38.8738,-77.1055,RentLingo,1577358561
1255,5668613880,housing/rent/apartment,Studio apartment 1351 N Rolfe St,"This unit is located at 1351 N Rolfe St, Arlin...",Pool,1.0,2.0,USD,No,Thumbnail,...,"$2,150",Monthly,535,1351 N Rolfe St,Arlington,VA,38.8738,-77.1055,RentLingo,1577358561
1568,5668613955,housing/rent/apartment,Studio apartment 505 18th Street South,"This unit is located at 505 18th Street South,...",Parking,1.0,2.0,USD,No,Thumbnail,...,"$1,953",Monthly,568,505 18th St South,Arlington,VA,38.8738,-77.1055,RentLingo,1577358567
3185,5509079183,housing/rent/apartment,Discover a new Aire Apartments in Arlington.,The location of this community Tharp. in the a...,Pool,1.0,2.0,USD,No,Thumbnail,...,$814,Monthly,690,NaN,Arlington,TX,32.7220,-97.0830,RentDigs.com,1568772228
